### wsb_picker sample usage
Conclusion: I learned some web crawling and machine learning, but if you've somehow ended up here from Google looking for a way to crawl Reddit for stock picks, this might help you learn basic crawling, but DO NOT USE THIS MODEL'S PREDICTIONS, it needs better data

In [7]:
from datetime import date
import wsbCrawler as wsb
import ThreadsFinder
import PrintToCSV

thread_type = 'What'
#Download recent comments (if wanted), will otherwise use sample comments included
#NOTE: WILL TRY TO OPEN A CROME BROWSER WINDOW WITH CHROMIUM
"""
num_days_back = 2
url = 'https://www.reddit.com/r/wallstreetbets/search/?q=flair%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new'
threads = ThreadsFinder.pull_threads(thread_type, url, num_days_back, skip_first_days=1)
wsb.download_comments(threads, comm_limit=2000)
"""

"\nnum_days_back = 2\nurl = 'https://www.reddit.com/r/wallstreetbets/search/?q=flair%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new'\nthreads = ThreadsFinder.pull_threads(thread_type, url, num_days_back, skip_first_days=1)\nwsb.download_comments(threads, comm_limit=2000)\n"

In [2]:
#LOAD IN COMMENTS
comments_by_date, num_threads = wsb.pull_comments_from_json(verbose=False)

#WORDS TO COUNT
words = wsb.pull_words_top(comments_by_date, num_words=1000)
#print (words)

JSON comment files imported:  ['.\\comments\\q249th.json', '.\\comments\\q87iwg.json', '.\\comments\\qa6f3y.json', '.\\comments\\qaupdv.json', '.\\comments\\qbjku7.json', '.\\comments\\qc9j00.json', '.\\comments\\qd038x.json', '.\\comments\\qezvzm.json', '.\\comments\\qfp08o.json', '.\\comments\\qgete2.json']
Number of comments imported: 13080, for thread starting at time: 1635278425
Number of comments imported: 11858, for thread starting at time: 1635192017
Number of comments imported: 8190, for thread starting at time: 1635105643
Number of comments imported: 14728, for thread starting at time: 1634846426
Number of comments imported: 9126, for thread starting at time: 1634760025
Number of comments imported: 10569, for thread starting at time: 1634673633
Number of comments imported: 8611, for thread starting at time: 1634587225
Number of comments imported: 7377, for thread starting at time: 1634500840
Number of comments imported: 11376, for thread starting at time: 1634241627
Number of

In [3]:
#get daily returns and indicators on mentioned stocks
stocks_data, indicators = wsb.merge_returns_counts_and_indicators(comments_by_date, words, short=False, verbose=False)

#name the CSV file
lastday = date.fromtimestamp(comments_by_date[0][0])
firstday = date.fromtimestamp(comments_by_date[len(comments_by_date)-1][0])
csv_file = f'counts_{thread_type}--from{firstday}_to_{lastday}.csv'

PrintToCSV.counts_organized(csv_file, stocks_data, words, indicators)

Stock history range needed: 2021-10-05 to 2021-10-27
DAY:  2021-10-26
Stocks counted
- RBC: No data found, symbol may be delisted
problem getting stock data out: '2021-10-27' STOCK:  RBC
- SC: No data found, symbol may be delisted
problem getting stock data out: '2021-10-27' STOCK:  SC
- TDA: No data found, symbol may be delisted
problem getting stock data out: '2021-10-27' STOCK:  TDA
- AGC: No data found, symbol may be delisted
problem getting stock data out: '2021-10-27' STOCK:  AGC
- APR: No data found, symbol may be delisted
problem getting stock data out: '2021-10-27' STOCK:  APR
- BRPMW: No data found for this date range, symbol may be delisted
problem getting stock data out: '2021-10-27' STOCK:  BRPMW
- DCRCW: No data found, symbol may be delisted
problem getting stock data out: '2021-10-27' STOCK:  DCRCW
- VIAC: No data found, symbol may be delisted
problem getting stock data out: '2021-10-27' STOCK:  VIAC
- XLNX: No data found, symbol may be delisted
problem getting stock dat

In [1]:
#MAKE PREDICTIONS!
import Predictor as p
import random

#made from larger sample than the json files included
csv_file_train = 'counts_What--from2021-08-31_to_2021-10-19.csv'
rnd = random.randint(0, 10000)
#rnd = 0
alpha = 1
C = .1
print ('RND SEED:', rnd)

#initial learning on split data
scaler, X, y, X_train, X_test, y_train, y_test = p.setup_data_categorized(csv_file_train, rnd)
logReg = p.log_regress(X_train, X_test, y_train, y_test,C=C)
#nn = neural_net(X_train, X_test, y_train, y_test, alpha=alpha) #using sklearn

#Then with that model, make predictions on the stocks listed in another file
csv_file_predict = 'counts_What--from2021-10-20_to_2021-10-26.csv'
tickers = p.get_tickers(csv_file_predict)
scaler, X, y, X_train, X_test, y_train, y_test = p.setup_data_categorized(csv_file_predict, rnd, scaler=scaler)
print('LOG PREDICTIONS FINAL')
p.clf_predictions_8_cats(X, logReg, tickers, y=y, cutoff=0.55)
#print('NET PREDICTIONS FINAL')
#clf_predictions_8_cats(X, nn, tickers, y=y, cutoff=.01)

RND SEED: 7544
Data shape:  (7174, 1018)
Cateorized y: [1 4 6 ... 6 0 3]
scaled
Log regress start
Log regress, split data, complete, score: 0.22909698996655517
Data shape:  (1040, 1018)
Cateorized y: [3 2 3 ... 1 1 1]
scaled
LOG PREDICTIONS FINAL
SCORE::::  0.24711538461538463


In [2]:
#Use custom Pytorch mlp neural net
import torchMLP as mlp
nn = mlp.batch_train(X_train, X_test, y_train, y_test, batch_size=64, epochs=20, lr=1e-2,
                     hidden_layer_sizes=(10000,3000,2000,1000,500), verbose=True, weight_decay=1e-5)

Device: cuda
Sequential(
  (input_layer): Linear(in_features=1017, out_features=10000, bias=True)
  (activation_input): ReLU()
  (hidden_layer_1): Linear(in_features=10000, out_features=3000, bias=True)
  (activation_1): ReLU()
  (hidden_layer_2): Linear(in_features=3000, out_features=2000, bias=True)
  (activation_2): ReLU()
  (hidden_layer_3): Linear(in_features=2000, out_features=1000, bias=True)
  (activation_3): ReLU()
  (hidden_layer_4): Linear(in_features=1000, out_features=500, bias=True)
  (activation_4): ReLU()
  (output): Linear(in_features=500, out_features=8, bias=True)
)
Epoch 0...
Epoch 0: Train loss: 2.084 accuracy: 0.060, Test loss: 2.080 accuracy: 0.035
Epoch 1: Train loss: 2.076 accuracy: 0.242, Test loss: 2.073 accuracy: 0.238
Epoch 2: Train loss: 2.069 accuracy: 0.256, Test loss: 2.066 accuracy: 0.238
Epoch 3: Train loss: 2.062 accuracy: 0.256, Test loss: 2.059 accuracy: 0.238
Epoch 4: Train loss: 2.056 accuracy: 0.256, Test loss: 2.053 accuracy: 0.238
Epoch 5: Tra